In [45]:
import numpy as np
import pandas as pd
import json

In [74]:
df = pd.read_csv('coords.csv', index_col=0)

In [75]:
# df['Destination x'] = None
# df['Destination y'] = None

In [76]:
with open('TRAVELNAME_COORDS.json', 'r') as f:
    travelname = json.loads(f.read())

with open('DESTINATION_COORDS.json', 'r') as f:
    destination = json.loads(f.read())

In [81]:
%%time

def maping_for_travelname(row):
    if row['Trip Name'] in travelname:
        row['x'] = travelname[row['Trip Name']]['x']
        row['y'] = travelname[row['Trip Name']]['y']
    return row

df = df.apply(maping_for_travelname, axis=1)

CPU times: user 15.5 s, sys: 144 ms, total: 15.6 s
Wall time: 15.6 s


In [82]:
%%time

def maping_for_destination(row):
    if row['Destination City'] in destination:
        row['x'] = destination[row['Destination City']]['x']
        row['y'] = destination[row['Destination City']]['y']
    return row

df = df.apply(maping_for_destination, axis=1)

CPU times: user 15.5 s, sys: 164 ms, total: 15.7 s
Wall time: 15.7 s


In [83]:
df[df['x'].isna()]

,Trip Name,Destination City,x,y,temp_addr
1950,Cyprus S21 (hotel only) RU,NaN,NaN,NaN,NaN
2143,Домбай-Приэльбрусье GDS MOW,NaN,NaN,NaN,NaN
2144,Домбай-Приэльбрусье GDS MOW,NaN,NaN,NaN,NaN
2166,SL TOUR Burgas/Varna MOW,NaN,NaN,NaN,NaN
2182,Классическая Италия MOW,NaN,NaN,NaN,NaN
...,...,...,...,...,...
209233,Калининград экскурсионные туры GDS MOW,NaN,NaN,NaN,NaN
209272,UAE Hotel Only part 3,NaN,NaN,NaN,NaN
209294,UAE Hotel Only part 3,NaN,NaN,NaN,NaN
209297,Maldives MOW GDS (Аэрофлот) воскресенье,NaN,NaN,NaN,NaN


In [84]:
df.to_csv('new_coords.csv')

In [88]:
df[['x', 'y']].drop_duplicates()

,x,y
0,30.701659,36.885843
2,35.302075,39.056249
3,30.525903,36.501248
9,31.998754,36.536567
11,34.051907,35.015774
...,...,...
191574,-0.553551,39.555550
193280,29.913441,58.041300
198358,33.232608,63.621324
207962,-3.695103,36.737532


# TO

In [133]:
newest_coords = pd.read_csv('newest_coords.csv', index_col=0)
xy = newest_coords[['x', 'y']]

In [109]:
from opencage.geocoder import OpenCageGeocode
key = '0e967af8a6bc4be7b9bcb0cc4d487d27'
geocoder = OpenCageGeocode(key)

In [121]:
def get_info(lat, lng)
    g = geocoder.reverse_geocode(lat=lat, lng=lng)

In [238]:
def prepare_info(g):
    info_d = {}
    
    d = g[0]
    
    annotations = d['annotations']
    info_d['MGRS'] = annotations['MGRS']
    info_d['currency'] = annotations['currency']['iso_code']
    info_d['flag'] = annotations['flag']
    info_d['roadinfo'] = annotations['roadinfo']['drive_on']
    info_d['timezone_name'] = annotations['timezone']['name']
    info_d['timezone_shortname'] = annotations['timezone']['short_name']
    
    components = d['components']
    info_d['continent'] = components['continent']
    
    if 'city' in components:
        city = components['city']
    elif 'town' in components:
        city = components['town']
    elif 'village' in components:
        city = components['village']
    elif 'county' in components:
        city = components['county']
    elif 'region' in components:
        city = components['region']
    elif 'municipality' in components:
        city = components['municipality']
    elif 'state' in components:
        city = components['state']
    else:
        city = None
    info_d['city'] = city
    
    info_d['country'] = components['country']
    
    info_d['confidence'] = d['confidence']
    
    geometry = d['geometry']
    info_d['lat'] = geometry['lat']
    info_d['lng'] = geometry['lng']   
    
    return info_d

In [239]:
with open('geo_from.json', 'r') as f:
    _from = json.loads(f.read())

with open('geo_to.json', 'r') as f:
    _to = json.loads(f.read())

In [240]:
preprocesed_from_dict = {' '.join(coords): prepare_info(data) for _, coords, data in _from if not coords is None}

In [218]:
pd.DataFrame(preprocesed_from_dict).T

,MGRS,currency,flag,roadinfo,timezone_name,timezone_shortname,continent,city,country,confidence,lat,lng
59.938951 30.315635,36VUM5003447663,RUB,🇷🇺,right,Europe/Moscow,MSK,Europe,Saint Petersburg,Russia,9,59.939043,30.315816
55.753215 37.622504,37UDB1329179347,RUB,🇷🇺,right,Europe/Moscow,MSK,Europe,Moscow,Russia,9,55.752044,37.6185
56.838011 60.597465,41VLD5342701988,RUB,🇷🇺,right,Asia/Yekaterinburg,+05,Europe,Yekaterinburg Municipality,Russia,10,56.83856,60.597313
55.796127 49.106405,39UUB8129685030,RUB,🇷🇺,right,Europe/Moscow,MSK,Europe,Kazan,Russia,9,55.796293,49.106535
53.195873 50.100193,39UVU3989494365,RUB,🇷🇺,right,Europe/Samara,+04,Europe,Samara,Russia,9,53.19522,50.100312
...,...,...,...,...,...,...,...,...,...,...,...,...
41.078385 1.140878,31TCF4382749125,EUR,🇪🇸,right,Europe/Madrid,CEST,Europe,Salou,Spain,9,41.078397,1.140885
25.805018 55.972714,40RCP9701654494,AED,🇦🇪,right,Asia/Dubai,+04,Asia,Ras Al Khaimah,United Arab Emirates,9,25.805028,55.972684
48.320515 40.268923,37UEP9406752703,RUB,🇷🇺,right,Europe/Moscow,MSK,Europe,Kamensk-Shakhtinsky,Russia,9,48.320521,40.268924
36.714834 -4.449463,30SUF7055764205,EUR,🇪🇸,right,Europe/Madrid,CEST,Europe,Málaga,Spain,9,36.714728,-4.449355


In [259]:
# preprocesed_to_dict = {' '.join(coords): prepare_info(data) for coords, data in _to if not np.isnan(coords[0])}

In [268]:
preprocesed_to_dict = {}

for coords, data in _to:
    try:
        preprocesed_to_dict[' '.join(map(str, coords))] = prepare_info(data)
    except Exception as e:
        print(e)
        pprint(data)

'currency'
[{'annotations': {'DMS': {'lat': "45° 10' 46.85880'' N",
                          'lng': "13° 3' 4.74120'' E"},
                  'MGRS': '33TUL4689404758',
                  'Maidenhead': 'JN65me63dc',
                  'Mercator': {'x': 1452865.963, 'y': 5619532.971},
                  'OSM': {'note_url': 'https://www.openstreetmap.org/note/new#map=16/45.17968/13.05132&layers=N',
                          'url': 'https://www.openstreetmap.org/?mlat=45.17968&mlon=13.05132#map=16/45.17968/13.05132'},
                  'geohash': 'u2130bxwtxr6cg19xk5n',
                  'qibla': 128.37,
                  'roadinfo': {'drive_on': 'right', 'speed_in': 'km/h'},
                  'sun': {'rise': {'apparent': 1616907420,
                                   'astronomical': 1616901360,
                                   'civil': 1616905620,
                                   'nautical': 1616903520},
                          'set': {'apparent': 1616952600,
                         

In [261]:
preprocesed_from_dict

{'59.938951 30.315635': {'MGRS': '36VUM5003447663',
  'currency': 'RUB',
  'flag': '🇷🇺',
  'roadinfo': 'right',
  'timezone_name': 'Europe/Moscow',
  'timezone_shortname': 'MSK',
  'continent': 'Europe',
  'city': 'Saint Petersburg',
  'country': 'Russia',
  'confidence': 9,
  'lat': 59.9390432,
  'lng': 30.3158163},
 '55.753215 37.622504': {'MGRS': '37UDB1329179347',
  'currency': 'RUB',
  'flag': '🇷🇺',
  'roadinfo': 'right',
  'timezone_name': 'Europe/Moscow',
  'timezone_shortname': 'MSK',
  'continent': 'Europe',
  'city': 'Moscow',
  'country': 'Russia',
  'confidence': 9,
  'lat': 55.7520439,
  'lng': 37.6185003},
 '56.838011 60.597465': {'MGRS': '41VLD5342701988',
  'currency': 'RUB',
  'flag': '🇷🇺',
  'roadinfo': 'right',
  'timezone_name': 'Asia/Yekaterinburg',
  'timezone_shortname': '+05',
  'continent': 'Europe',
  'city': 'Yekaterinburg Municipality',
  'country': 'Russia',
  'confidence': 10,
  'lat': 56.8385597,
  'lng': 60.5973125},
 '55.796127 49.106405': {'MGRS': '39U

In [269]:
with open('from_matching.json', 'w') as f:
    json.dump(preprocesed_from_dict, f)

with open('to_matching.json', 'w') as f:
    json.dump(preprocesed_to_dict, f)